# Imports

In [ ]:
!pip install ffmpeg-python
!pip install av
!pip install -q -U google-generativeai
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate
!pip install scenedetect

In [ ]:
import os
import cv2
import time
import torch
import random
import ffmpeg
import warnings
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
import soundfile as sf
from json import loads,dumps
#from pydub import AudioSegment
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy.signal import resample
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from scenedetect import open_video, VideoStreamCv2, SceneManager
from scenedetect.detectors import ContentDetector

# Data Loading

In [ ]:
# Extracting all required ids 

labels_df = pd.read_csv('/kaggle/input/youtube-data/all_unique_codes3.csv') 
transcriptions_df = pd.read_csv('/kaggle/input/youtube-data/Translated-transcriptions.csv')

# Merging on different key names
df = pd.merge(labels_df, transcriptions_df, left_on='Video link', right_on='Video Id')
df 

In [ ]:
# Extracting video ids and primary labels

video_ids = list(df['Video Id'])
primary_labels = list(df['Primary Tag'])
all_transcriptions = list(df['Transcription'])

In [ ]:
len(video_ids) 

In [ ]:
# Extracting data from transcripts

transcriptions = []
lengths = []

for (i, id_) in enumerate(video_ids):
    transcriptions.append(all_transcriptions[i].split("chunks")[0])
    lengths.append(len(all_transcriptions[i].split("chunks")[0]))

In [ ]:
available_ad_ids = os.listdir('/kaggle/input/youtube-data/Ads/Ads') 
len(available_ad_ids)

**Extracting Images** 

In [ ]:
def detect_scenes(video_path, threshold = 30):
    """Detect scenes in a video and return scene start and end frames."""
    scene_list = []
    while len(scene_list) < 6 and threshold > 0:
        threshold //= 2
    
        video = open_video(video_path)
        scene_manager = SceneManager()
        scene_manager.add_detector(ContentDetector(threshold=threshold))
    
        scene_manager.detect_scenes(video)
        scene_list = scene_manager.get_scene_list()
    
    return scene_list


def get_top_n_longest_scenes(scene_list, n):
    '''Return the top n longest scenes with start and end frame indices.'''
    scene_durations = [(start, end - start) for start, end in scene_list]
    scene_durations.sort(key=lambda x: x[1], reverse=True)

    # Top n longest scenes with start and end frame indices
    longest_scenes = [(start, start + duration) for start, duration in scene_durations[:n]]
    return longest_scenes


def sort_scenes_by_frame(scenes_list):
    '''Sort scenes by their start frame number.'''
    sorted_scenes = sorted(scenes_list, key=lambda scene: scene[0].get_frames())
    return sorted_scenes


def get_num_grids(video_path):
    '''Get number of grids to be created'''
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = total_frames / fps

    # Calculate number of grids based on the duration
    duration = round(duration, 2)
    if ((duration // 60) + 1) <= 5:
        return int(((duration // 60) + 1))
    else:
        return 5


def extract_k_frames_from_scene(video_path, scene, k):
    '''Extract k frames evenly spaced from each scene.'''
    # Extract frame numbers from scene start and end
    start_frame = scene[0].get_frames() + 1
    end_frame = scene[1].get_frames() - 1

    # Create k equally spaced frame indices within the scene's range
    frame_indices = np.linspace(start_frame, end_frame, k, dtype=int)
    
    cap = cv2.VideoCapture(video_path)
    frames = []

    # Extract frames from calculated indices
    for frame_no in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_no)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    
    cap.release()
    return frames


def create_image_grid(frames, grid_size=(1000, 1000)):
    '''Arrange 6 frames into a 3x2 grid and resize to the specified grid size.'''
    # Ensure all frames have the same size for concatenation
    frames = [cv2.resize(frame, (640, 360)) for frame in frames]  # Resize to a common size like 640x360
    rows = [np.concatenate(frames[i:i+2], axis=1) for i in range(0, 6, 2)]
    image_grid = np.concatenate(rows, axis=0)
    
    return np.array(Image.fromarray(image_grid).resize(grid_size))

In [ ]:
def get_images(video_path, n=6):
    ''' 1. Detect scenes
        2. Get k; where k = num_grids
        3. Get the 6 longest scenes
        4. Sort scenes wrt frame numbers
        5. Extract n * k frames
        6. Create k image grids of n frames each
     '''
    scene_list = detect_scenes(video_path)
    k = get_num_grids(video_path)
    longest_scenes = get_top_n_longest_scenes(scene_list, n*k)
    scenes = sort_scenes_by_frame(longest_scenes)

    frames = []
    for scene in scenes:
        frames.extend(extract_k_frames_from_scene(video_path, scene, 1))

    grids = []
    for i in range(k):
        start_idx = i * n
        end_idx = start_idx + n
        grid_frames = frames[start_idx:end_idx]
        grid = create_image_grid(grid_frames, grid_size=(1000, 1000))
        grids.append(grid)

    return grids

**Defining Model** 

In [ ]:
key_ = '' # add key here 

In [ ]:
import google.generativeai as genai 
os.environ["API_KEY"] = key_ 
genai.configure(api_key=os.environ["API_KEY"]) 

In [ ]:
print(torch.cuda.is_available())

In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash", system_instruction="You are an expert content moderator.") 

In [ ]:
class LLM_Output(typing.TypedDict):
    label: str
    language: list[str]
    response: str

**Prompt** 

In [ ]:
prompt =  """ A video is inappropriate for children if it contains physical violence (cartoonish, realistic, inclusion of gore),
  interpersonal violence (such as bullying, pranks, meanness, humor at expense of others, belittling ,controlling, talking down to, or manipulating others),
  self-harm or suicide,
  elements of scariness (horror, scary themes, creepy or suspenseful themes, loud and violent content, zombies, skeletons, scary masks, scary clowns, blood, fire in dangerous settings,
  mentally distressing scenes (car crashes, medical procedures etc),
  extreme stunts (depictions of life endangering or high-risk physical activities/challenges),
  sexual content (innuendos, overt sexual behavior, nudity, sexual themes, sexual suggestiveness),
  discrimination (based on race/ethnicity/gender/ability),
  offensive language (from mild words such as "idiot" and "stupid" to severe swearing, hate speech or signs),
  drinking, drugs, smoking, gambling, crude humour,
  inappropriate clothing (with significant skin exposure),
  death,
  intimacy (sensual physical contact, marriage, romance, dating and relationships, apparent romantic conduct between subjects/actors).
  romantic songs and music.

  You have been given an image grid (3 x 2) representing a video and it's audio transcription.
  Consider that you are a Parent. Based on both the visual content and audio transcription, is the video ad provided inappropriate for children under 9 based on this definition?
  If it is, label it as "inappropriate".
  If it is not, label it as "appropriate".

  Justify your conclusion.

  Also, mention what language is being spoken if there is any.

  Make the outputs in JSON format.

  """

**Running Model on Dataset** 

In [ ]:
len(video_ids) 

In [ ]:
ids = []
predicted_labels = []
languages = []
responses = []
ground_truths = []
remaining = []

img_dir = '/kaggle/working/Images'
if not os.path.exists(img_dir):
    os.makedirs(img_dir)
    
for i in range(len(video_ids)): 

    if video_ids[i] in available_ad_ids:

        try:
            contents_of_ad = os.listdir('/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i]) 
            contents_of_ad.remove('audio.mp3') 
            video_path = '/kaggle/input/youtube-data/Ads/Ads/' + video_ids[i] + '/' + contents_of_ad[0] 

            # Extract multiple images representative of the video
            images = get_images(video_path) 

            # Save each image returned by extract_images_of_frames
            image_paths = []
            for idx, img in enumerate(images):
                # Convert NumPy array to PIL image
                image = Image.fromarray(img)
                
                # Save the image to a file
                image_name = f"{video_ids[i]}_{idx + 1}.png"
                image_path = os.path.join(img_dir, image_name)
                image.save(image_path)
                image_paths.append(image_path)
                print(image_path)
                
            # Display the images
            fig, axes = plt.subplots(1, len(images), figsize=(10, 3))
            if len(images) == 1:
                axes.imshow(images[0])
                axes.axis('off')
            else:
                for ax, img in zip(axes, images):
                    ax.imshow(img)
                    ax.axis('off')

            plt.tight_layout()
            plt.show()

            # Upload images and handle potential errors
            uploaded_files = []
            try:
                for image_path in image_paths:
                    uploaded_file = genai.upload_file(path=image_path, resumable=False)
                    uploaded_files.append(uploaded_file)
            except Exception as e:
                print(f"Error uploading images: {e}")
                remaining.append(video_ids[i])
                continue

            # Check if all images have uploaded
            try:
                for uploaded_file in uploaded_files:
                    while uploaded_file.state.name == "PROCESSING":
                        print('.', end='')
                        time.sleep(10)
                        uploaded_file = genai.get_file(uploaded_file.name)
                    if uploaded_file.state.name == "FAILED":
                        raise ValueError(uploaded_file.state.name)
            except Exception as e:
                print(f"Error during image processing: {e}")
                remaining.append(video_ids[i])
                continue

            # Make inference with audio and image URIs
            audio = transcriptions[i]
            try: 
                inputs_ = [audio] 
                inputs_.extend(uploaded_files) 
                inputs_.extend([prompt]) 
                response = model.generate_content(inputs_,
                                                  generation_config=genai.GenerationConfig(
                                                      response_mime_type="application/json",
                                                      response_schema=LLM_Output, 
                                                      temperature=0.0), 
                                                  safety_settings={
                                                    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                                                    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
                                                  })
            except Exception as e:
                print(f"Error making inference: {e}")
                remaining.append(video_ids[i])
                continue
            try:
                print("Completed for video number:", i, ' ', video_ids[i])

                dictionary = loads(response.text)
                print('True Label:', primary_labels[i], 'Response:', dictionary)

                ids.append(video_ids[i])
                predicted_labels.append(dictionary['label'])
                languages.append(dictionary['language'])
                responses.append(dictionary['response'])
                ground_truths.append(primary_labels[i]) 
                
            except Exception as e:
                print(f"Error processing response.text: {e}")
                remaining.append(video_ids[i])
                continue

        except Exception as e:
            print(f"Unexpected error: {e}")
            remaining.append(video_ids[i])
            continue

        time.sleep(20)

In [ ]:
len(ids)

In [ ]:
# At the end, print remaining videos 

print("Remaining videos with errors:", remaining) 

In [ ]:
for i in range(len(responses)): 
    print('True Label: ', ground_truths[i], '\tPrediction: ', predicted_labels[i]) 

In [ ]:
new_df = pd.DataFrame({
    'Video Id': ids,
    'Primary Label': ground_truths,
    'Predicted Label': predicted_labels,
    'Response': responses, 
    'Languages': languages 
})

new_df.head() 

In [ ]:
results_dir = '/kaggle/working/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [ ]:
new_df.to_csv('/kaggle/working/results/DAVSP_Gemini-1.5-Flash.csv', index=False)

In [ ]:
# Changing to binary lists 

predictions = [1 if pred == 'inappropriate' else 0 for pred in predicted_labels] 
ground_truths = [1 if label == 'inappropriate' else 0 for label in ground_truths] 

In [ ]:
# Obtaining classification report 
from sklearn.metrics import classification_report 

report = classification_report(ground_truths, predictions) 
print(report) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ground_truths, predictions)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Child Directed', 'Inapproriate'], yticklabels=['Child Directed', 'Inapproriate'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()